# 1. dataset 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from time import time 

In [ ]:
dataset = pd.read_csv('/content/drive/MyDrive/OBGY_modeling/project1_model_dataset.csv', encoding = "euc-kr")
dataset.head()

,age,twin,bmi,전자간증,PIH임신중고혈압,고혈압,산과력_출산력P,산과력_출산력A,수축억제제,저체중아,태아성장지연,태반조기박리,부인과수술력,자궁봉축술,입원총기간,입원횟수,첫투약시기,outcome
0,38,0,27.70,0,0,1,1,1,1,0,0,0,0,0,2,0,0,37.0
1,33,0,21.45,0,0,0,0,3,0,0,0,0,0,0,9,1,0,30.4
2,22,0,30.85,0,0,0,0,0,0,0,0,0,0,0,7,1,0,38.0
3,34,0,23.65,0,0,0,0,0,0,0,1,0,0,0,2,0,0,38.0
4,38,0,23.19,0,0,0,0,0,1,0,0,0,0,0,2,0,0,37.0


In [ ]:
# Checking for null values
print(dataset.info())

# Checking for outliers
print(dataset.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16109 entries, 0 to 16108
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        16109 non-null  int64  
 1   twin       16109 non-null  int64  
 2   bmi        16109 non-null  float64
 3   전자간증       16109 non-null  int64  
 4   PIH임신중고혈압  16109 non-null  int64  
 5   고혈압        16109 non-null  int64  
 6   산과력_출산력P   16109 non-null  int64  
 7   산과력_출산력A   16109 non-null  int64  
 8   수축억제제      16109 non-null  int64  
 9   저체중아       16109 non-null  int64  
 10  태아성장지연     16109 non-null  int64  
 11  태반조기박리     16109 non-null  int64  
 12  부인과수술력     16109 non-null  int64  
 13  자궁봉축술      16109 non-null  int64  
 14  입원총기간      16109 non-null  int64  
 15  입원횟수       16109 non-null  int64  
 16  첫투약시기      16109 non-null  int64  
 17  outcome    16109 non-null  float64
dtypes: float64(2), int64(16)
memory usage: 2.2 MB
None
                age          twin  

In [ ]:
# factor로 변환
# 성별, twin, 전자간증, PIH임신중고혈압, 고혈압, 산과력_출산력A, 산과력_출산력P, 응급실_횟수, 입원_횟수, 첫투약시기, outcome
  
dataset['twin'] = dataset['twin'].astype('category')
dataset['전자간증'] = dataset['전자간증'].astype('category')
dataset['PIH임신중고혈압'] = dataset['PIH임신중고혈압'].astype('category')
dataset['고혈압'] = dataset['고혈압'].astype('category')
dataset['산과력_출산력A'] = dataset['산과력_출산력A'].astype('category')
dataset['산과력_출산력P'] = dataset['산과력_출산력P'].astype('category')
dataset['수축억제제'] = dataset['수축억제제'].astype('category')
dataset['저체중아'] = dataset['저체중아'].astype('category')
dataset['태아성장지연'] = dataset['태아성장지연'].astype('category')
dataset['태반조기박리'] = dataset['태반조기박리'].astype('category')
dataset['부인과수술력'] = dataset['부인과수술력'].astype('category')
dataset['자궁봉축술'] = dataset['자궁봉축술'].astype('category')
dataset['입원횟수'] = dataset['입원횟수'].astype('category')
dataset['첫투약시기'] = dataset['첫투약시기'].astype('category')

In [ ]:
dataset.dtypes

age             int64
twin         category
bmi           float64
전자간증         category
PIH임신중고혈압    category
고혈압          category
산과력_출산력P     category
산과력_출산력A     category
수축억제제        category
저체중아         category
태아성장지연       category
태반조기박리       category
부인과수술력       category
자궁봉축술        category
입원총기간           int64
입원횟수         category
첫투약시기        category
outcome       float64
dtype: object

In [ ]:
dataset.isna().sum()

age          0
twin         0
bmi          0
전자간증         0
PIH임신중고혈압    0
고혈압          0
산과력_출산력P     0
산과력_출산력A     0
수축억제제        0
저체중아         0
태아성장지연       0
태반조기박리       0
부인과수술력       0
자궁봉축술        0
입원총기간        0
입원횟수         0
첫투약시기        0
outcome      0
dtype: int64

# 2. 데이터 전처리 

In [ ]:
from sklearn.model_selection import train_test_split
np.random.seed(0)
df_train, df_test = train_test_split(dataset,train_size =0.75, test_size = 0.25, random_state = 100)

In [ ]:
# 전처리 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# numeric 변수 scale 
sc = StandardScaler()  #평균 0 , 분산 1로 조정
scaler = MinMaxScaler()

num_vars = ['age','입원총기간','bmi','outcome']
df_train[num_vars] = scaler.fit_transform(df_train[num_vars])
df_test[num_vars] = scaler.fit_transform(df_test[num_vars])

df_train

,age,twin,bmi,전자간증,PIH임신중고혈압,고혈압,산과력_출산력P,산과력_출산력A,수축억제제,저체중아,태아성장지연,태반조기박리,부인과수술력,자궁봉축술,입원총기간,입원횟수,첫투약시기,outcome
4332,0.433333,0,0.280100,0,0,0,0,0,0,0,0,0,0,0,0.008403,0,0,0.772727
9264,0.566667,0,0.275277,0,0,0,0,2,0,0,0,0,0,0,0.008403,0,0,0.290909
6616,0.566667,0,0.363701,0,0,0,0,0,0,0,0,0,0,0,0.008403,0,0,0.363636
10716,0.633333,1,0.325116,0,0,0,0,2,0,0,0,0,0,0,0.000000,0,0,0.681818
13695,0.433333,0,0.241515,0,0,0,0,0,0,0,0,0,0,0,0.008403,0,0,0.863636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12119,0.366667,0,0.419078,0,0,0,0,0,0,0,0,0,0,0,0.008403,0,0,0.400000
8039,0.633333,0,0.222401,0,0,0,0,0,0,0,0,0,0,0,0.008403,0,0,0.772727
14147,0.700000,1,0.374062,0,0,0,0,0,0,0,0,0,0,0,0.008403,0,0,0.681818
6936,0.766667,0,0.247946,0,0,0,0,1,0,0,0,0,0,0,0.008403,0,0,0.863636


In [ ]:
# X_train, y_train 나누기 
X_train = df_train.iloc[:, :-1]
y_train = df_train.iloc[:, -1]

X_test = df_test.iloc[:, :-1]
y_test = df_test.iloc[:, -1]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(12081, 17) (12081,) (4028, 17) (4028,)


# 3. regression - benchmark model

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.linear_model import ARDRegression,BayesianRidge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
import ast #convert string to function
import statsmodels.api as sm 

In [ ]:
X_train_lm = sm.add_constant(X_train)

lr_1 = sm.OLS(y_train, X_train_lm).fit()

print(lr_1.summary())

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.178
Model:                            OLS   Adj. R-squared:                  0.177
Method:                 Least Squares   F-statistic:                     163.6
Date:                Fri, 04 Nov 2022   Prob (F-statistic):               0.00
Time:                        04:17:21   Log-Likelihood:                 5354.0
No. Observations:               12081   AIC:                        -1.067e+04
Df Residuals:                   12064   BIC:                        -1.055e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8060      0.008    100.235      0.0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


# Recursive Feature Elimination

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
# Running RFE with the output number of the variable equal to 10
if __name__ == "__main__":
    
    lr = LinearRegression()

    rfe = RFE(lr, step = 10) 
    fit = rfe.fit(X_train, y_train)
    print("Features: {features}".format(features=X_train.columns))
    print("Num Features: {number_features}".format(number_features=fit.n_features_))
    print("Selected Features: {support}".format(support=fit.support_))
    print("Feature Ranking: {ranking}".format(ranking=fit.ranking_))

    selected_columns = [column for column, selected in zip(X_train.columns, fit.support_) if selected]
    print("Selected columns: {selected}".format(selected = selected_columns))


Features: Index(['age', 'twin', 'bmi', '전자간증', 'PIH임신중고혈압', '고혈압', '산과력_출산력P',
       '산과력_출산력A', '수축억제제', '저체중아', '태아성장지연', '태반조기박리', '부인과수술력', '자궁봉축술',
       '입원총기간', '입원횟수', '첫투약시기'],
      dtype='object')
Num Features: 8
Selected Features: [False  True  True  True False False False False False False  True  True
 False  True  True False  True]
Feature Ranking: [2 1 1 1 2 2 2 2 2 2 1 1 2 1 1 2 1]
Selected columns: ['twin', 'bmi', '전자간증', '태아성장지연', '태반조기박리', '자궁봉축술', '입원총기간', '첫투약시기']


In [ ]:
# Creating X_test dataframe with RFE selected variables
col = ['twin', 'bmi', '전자간증', '태아성장지연', '태반조기박리', '자궁봉축술', '입원총기간', '첫투약시기']
X_train_rfe = X_train[col]

# Adding a constant variable 
X_train_rfe = sm.add_constant(X_train_rfe)

lm = sm.OLS(y_train,X_train_rfe).fit()   # Running the linear model

print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.174
Model:                            OLS   Adj. R-squared:                  0.174
Method:                 Least Squares   F-statistic:                     318.3
Date:                Fri, 04 Nov 2022   Prob (F-statistic):               0.00
Time:                        04:17:21   Log-Likelihood:                 5323.7
No. Observations:               12081   AIC:                        -1.063e+04
Df Residuals:                   12072   BIC:                        -1.056e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7946      0.006    133.785      0.0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
# 자궁봉축술 제거 
X_train_new = X_train_rfe.drop(["자궁봉축술"], axis = 1)

# Adding a constant variable 
X_train_lm = sm.add_constant(X_train_new)

lm = sm.OLS(y_train,X_train_lm).fit()   # Running the linear model

print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                outcome   R-squared:                       0.174
Model:                            OLS   Adj. R-squared:                  0.173
Method:                 Least Squares   F-statistic:                     362.8
Date:                Fri, 04 Nov 2022   Prob (F-statistic):               0.00
Time:                        04:17:21   Log-Likelihood:                 5320.8
No. Observations:               12081   AIC:                        -1.063e+04
Df Residuals:                   12073   BIC:                        -1.057e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7944      0.006    133.739      0.0

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
# VIF = 다중공선성 확인
X_train_new = X_train_new.drop(['const'], axis=1)

# Calculate the VIFs for the new model
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
X = X_train_new
vif['Features'] = X.columns
vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

,Features,VIF
1,bmi,1.29
5,입원총기간,1.20
0,twin,1.13
6,첫투약시기,1.11
2,전자간증,1.07
3,태아성장지연,1.00
4,태반조기박리,1.00


In [ ]:
# predict - test data

# Creating X_test_new dataframe by dropping variables from X_test
X_test_new = X_test[X_train_new.columns]

# Adding a constant variable 
X_test_new = sm.add_constant(X_test_new)

# Making predictions
y_pred = lm.predict(X_test_new)

r2_score(y_true = y_test, y_pred = y_pred) #0.17

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


0.12188487337576592

In [ ]:
import ast 
# model
def my_regressor(df_X, df_y):
  linear = LinearRegression()
  ridge, lasso, elasticnet = Ridge(), Lasso(), ElasticNet()
  ardr_linear, baysian_ridge = ARDRegression(), BayesianRidge()
  xgboost = XGBRegressor(enable_categorical=True)
  svc = SVR(kernel = 'rbf')
  random = RandomForestRegressor(n_estimators = 10, random_state = 0)
  decision = DecisionTreeRegressor(random_state = 0)  

  my_model_list = ['linear','ridge','lasso','elasticnet','ardr_linear','baysian_ridge',
                   'xgboost','svc','random','decision']
  score_dic = dict()

  for model_nm in my_model_list:
    scores = cross_val_score(eval(model_nm), df_X, df_y,scoring = "neg_mean_squared_error") #cv =5
    rmse_score = np.sqrt(-scores)
    rmse_sm = rmse_score.mean()
    score_dic[model_nm] = rmse_sm
  
  score_dic = sorted(score_dic.items(),key = lambda t :t[1])
  
  score_dic = pd.DataFrame()
  return score_dic


my_regressor(X_train, y_train)

# Machine Learning Modeling

In [ ]:
# Function to fit the regressor and record performance metrics
def pipeline(reg, X_train, y_train, X_test, y_test, **kwargs):
    
    # Dictionary to hold properties of Models
    reg_props = {}
    
    # Initialize and fit the regressor, and time taken
    regressor = reg(**kwargs)
    start = time()
    regressor.fit(X_train, y_train)
    end = time()
    
    # Store the metrics for the regressor
    reg_props["name"] = reg.__name__
    reg_props["train_time"] = end - start
    reg_props["train_score"] = regressor.score(X_train, y_train)
    reg_props["test_score"] = regressor.score(X_test, y_test)
    reg_props["rmse"] = np.sqrt(mean_squared_error(y_test, regressor.predict(X_test)))
    reg_props["MAE"] = mean_absolute_error(y_test, regressor.predict(X_test))
    
    return reg_props

In [ ]:
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.neural_network import MLPRegressor

# Function to execute each algorithm through the pipeline
def execute_pipeline():
    # Create the list of algorithms
    regressors = [
        LinearRegression,
        Ridge,
        Lasso,
        ElasticNet,
        SVR,
        KNeighborsRegressor,
        RandomForestRegressor,
        GradientBoostingRegressor,
        ExtraTreesRegressor,
            ]
    
    # To store the properties for each regressor
    props = []
    
    """
    Iterate thorugh the list of regressors,
    passing each thorugh the pipeline and
    storing its properites
    """ 
    for reg in regressors:
        properites = pipeline(reg, X_train, y_train, X_test, y_test)
        props.append(properites)
        
    return props

In [ ]:
def get_properties():
    # Obtain the properties after executing the pipeline
    properties = execute_pipeline()
    
    # Extract each individual property of the Regressors
    names = [prop["name"] for prop in properties]
    train_times = [prop["train_time"] for prop in properties]
    train_scores = [prop["train_score"] for prop in properties]
    test_scores = [prop["test_score"] for prop in properties]
    rmse_vals = [prop["rmse"] for prop in properties]
    mae_vals = [prop["MAE"] for prop in properties]

        
    # Create a DataFrame from these properties
    df = pd.DataFrame(index=names, 
                    data = {
                            "Training Times": train_times,
                            "Training Scores": train_scores,
                            "Testing Scores": test_scores,
                            "RMSE": rmse_vals,
                            "MAE":mae_vals
                      }
                  )
    
    return df

# Obain the properties in a structured DataFrame after executing the pipeline
properties = get_properties()
properties 

,Training Times,Training Scores,Testing Scores,RMSE,MAE
LinearRegression,0.011806,0.178322,0.126560,0.166851,0.137284
Ridge,0.004755,0.178300,0.126583,0.166849,0.137346
Lasso,0.006287,0.000000,-0.037270,0.181827,0.152295
ElasticNet,0.006191,0.000000,-0.037270,0.181827,0.152295
SVR,3.031691,0.280130,0.185081,0.161165,0.130035
KNeighborsRegressor,0.003046,0.409299,0.046301,0.174348,0.139217
RandomForestRegressor,3.074810,0.844639,0.048609,0.174137,0.141872
GradientBoostingRegressor,0.708934,0.294017,0.180059,0.161660,0.135224
ExtraTreesRegressor,1.665632,0.945004,0.030606,0.175777,0.140071
